In [1]:
import pandas as pd

In [42]:
stock_mentions = "../data/stock_mentions.parquet"
stock_mentions = pd.read_parquet(stock_mentions)
stock_mentions

,ticker,original_word,match_score,text,text_index,origin,original_index
0,FAST,fast,100.0,$INO rising fast,41,submission_title,41
1,TECH,TECH,100.0,Best TECH ETF?,179,submission_title,179
2,TECH,Tech,100.0,EqualOcean Presents: China Big Tech Crackdown ...,307,submission_title,307
3,COST,cost,100.0,Delta Airlines wants to increase its unvaccina...,427,submission_title,427
4,TECH,tech,100.0,Analysis into whether certain Chinese tech sto...,449,submission_title,449
...,...,...,...,...,...,...,...
2197,GOOG,GOOG,100.0,"Index fund such as FXAIX $300, TSLA $200, GOOG...",58695,comment,8909
2198,NVDA,NVDA,100.0,"Index fund such as FXAIX $300, TSLA $200, GOOG...",58695,comment,8909
2199,WELL,well,100.0,Cifr is poised to do very well in 2025.,58704,comment,8918
2200,TECH,tech,100.0,Their minimal invasive treatment sounds like i...,58730,comment,8944


In [3]:
from nlp_utils import score_text_gemini

with open("api_key.txt", "r") as f:
    api_key = f.read().strip()

In [ ]:
def score_row_gemini(row):
    score, tokens = score_text_gemini(row['text'], row['ticker'], api_key)
    print(f'scored {row['origin']} at index {row['original_index']}')
    return score

scores_series = stock_mentions.head(100).apply(score_row_gemini, axis=1)

ticker                        FAST
original_word                 fast
match_score                  100.0
text              $INO rising fast
text_index                      41
origin            submission_title
original_index                  41
Name: 0, dtype: object
scored submission_title at index 41


KeyboardInterrupt: 

In [ ]:
reduced_stock_mentions = stock_mentions.head(100)
reduced_stock_mentions['score'] = scores_series
reduced_stock_mentions

/var/folders/2n/ywnb5cc17zq6c5n2w2n7bkzw0000gn/T/ipykernel_15328/2621434444.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_stock_mentions['score'] = scores_series
/var/folders/2n/ywnb5cc17zq6c5n2w2n7bkzw0000gn/T/ipykernel_15328/2621434444.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_stock_mentions['tokens'] = tokens_series


,ticker,original_word,match_score,text,text_index,origin,original_index,score,tokens
0,FAST,fast,100.0,$INO rising fast,41,submission_title,41,1,92
1,TECH,TECH,100.0,Best TECH ETF?,179,submission_title,179,0,92
2,TECH,Tech,100.0,EqualOcean Presents: China Big Tech Crackdown ...,307,submission_title,307,0,98
3,COST,cost,100.0,Delta Airlines wants to increase its unvaccina...,427,submission_title,427,0,109
4,TECH,tech,100.0,Analysis into whether certain Chinese tech sto...,449,submission_title,449,0,98
...,...,...,...,...,...,...,...,...,...
95,META,META,100.0,META Summarized News for the week [Feb 19],5745,submission_title,5745,0,101
96,META,Meta,100.0,The Meta triangle wedge is finishing up,5923,submission_title,5923,0,95
97,AXON,AXON,100.0,AXON,5957,submission_title,5957,0,106
98,COST,Cost,100.0,Cost of Capital: Charlie Munger's Sarcastic Ta...,6155,submission_title,6155,0,105


In [15]:
reduced_stock_mentions.to_parquet("fine_tuning/data/reduced_stock_mentions.parquet")

In [28]:
reduced_stock_mentions = pd.read_parquet("fine_tuning/data/reduced_stock_mentions.parquet")
reduced_stock_mentions

,ticker,original_word,match_score,text,text_index,origin,original_index,score,tokens
0,FAST,fast,100.0,$INO rising fast,41,submission_title,41,1,92
1,TECH,TECH,100.0,Best TECH ETF?,179,submission_title,179,0,92
2,TECH,Tech,100.0,EqualOcean Presents: China Big Tech Crackdown ...,307,submission_title,307,0,98
3,COST,cost,100.0,Delta Airlines wants to increase its unvaccina...,427,submission_title,427,0,109
4,TECH,tech,100.0,Analysis into whether certain Chinese tech sto...,449,submission_title,449,0,98
...,...,...,...,...,...,...,...,...,...
95,META,META,100.0,META Summarized News for the week [Feb 19],5745,submission_title,5745,0,101
96,META,Meta,100.0,The Meta triangle wedge is finishing up,5923,submission_title,5923,0,95
97,AXON,AXON,100.0,AXON,5957,submission_title,5957,0,106
98,COST,Cost,100.0,Cost of Capital: Charlie Munger's Sarcastic Ta...,6155,submission_title,6155,0,105


In [29]:
from nlp_utils import system_text

def system_row(row):
    prompt = system_text(row['ticker'])
    return prompt

reduced_stock_mentions['system_text'] = reduced_stock_mentions.apply(system_row, axis=1)
reduced_stock_mentions

,ticker,original_word,match_score,text,text_index,origin,original_index,score,tokens,system_text
0,FAST,fast,100.0,$INO rising fast,41,submission_title,41,1,92,"You need to act as an expert linguist, trader ..."
1,TECH,TECH,100.0,Best TECH ETF?,179,submission_title,179,0,92,"You need to act as an expert linguist, trader ..."
2,TECH,Tech,100.0,EqualOcean Presents: China Big Tech Crackdown ...,307,submission_title,307,0,98,"You need to act as an expert linguist, trader ..."
3,COST,cost,100.0,Delta Airlines wants to increase its unvaccina...,427,submission_title,427,0,109,"You need to act as an expert linguist, trader ..."
4,TECH,tech,100.0,Analysis into whether certain Chinese tech sto...,449,submission_title,449,0,98,"You need to act as an expert linguist, trader ..."
...,...,...,...,...,...,...,...,...,...,...
95,META,META,100.0,META Summarized News for the week [Feb 19],5745,submission_title,5745,0,101,"You need to act as an expert linguist, trader ..."
96,META,Meta,100.0,The Meta triangle wedge is finishing up,5923,submission_title,5923,0,95,"You need to act as an expert linguist, trader ..."
97,AXON,AXON,100.0,AXON,5957,submission_title,5957,0,106,"You need to act as an expert linguist, trader ..."
98,COST,Cost,100.0,Cost of Capital: Charlie Munger's Sarcastic Ta...,6155,submission_title,6155,0,105,"You need to act as an expert linguist, trader ..."


In [37]:
json_data = [{"messages": [{"role": "system", "content": row["system_text"]}, {"role": "user", "content": row["text"]},{"role": "assistant", "content": row["score"]}]} for _, row in reduced_stock_mentions.iterrows()]

In [38]:
json_df = pd.DataFrame(json_data)
json_df

,messages
0,"[{'role': 'system', 'content': 'You need to ac..."
1,"[{'role': 'system', 'content': 'You need to ac..."
2,"[{'role': 'system', 'content': 'You need to ac..."
3,"[{'role': 'system', 'content': 'You need to ac..."
4,"[{'role': 'system', 'content': 'You need to ac..."
...,...
95,"[{'role': 'system', 'content': 'You need to ac..."
96,"[{'role': 'system', 'content': 'You need to ac..."
97,"[{'role': 'system', 'content': 'You need to ac..."
98,"[{'role': 'system', 'content': 'You need to ac..."


In [39]:
from sklearn.model_selection import train_test_split

# Split into 80% train, 20% temp (which will be split further)
train_data, temp_data = train_test_split(json_df, test_size=0.2, random_state=42)

# Split temp into 50% validation, 50% test (so each is 10% of total)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Convert DataFrames back to list format for JSON saving
train_json = train_data.to_dict(orient="records")
val_json = val_data.to_dict(orient="records")
test_json = test_data.to_dict(orient="records")

In [41]:
# Save train set
with open("fine_tuning/data/split/train.jsonl", "w", encoding="utf-8") as f:
    for item in train_json:
        f.write(json.dumps(item) + '\n')

# Save validation set
with open("fine_tuning/data/split/valid.jsonl", "w", encoding="utf-8") as f:
    for item in val_json:
        f.write(json.dumps(item) + '\n')

# Save test set
with open("fine_tuning/data/split/test.jsonl", "w", encoding="utf-8") as f:
    for item in test_json:
        f.write(json.dumps(item) + '\n')

In [58]:
from nlp_utils import score_text_local

# model_path = 'fine_tuning/models/fused_model'
model_path = 'mlx-community/Llama-3.2-3B-Instruct-4bit'

def score_row_local(row):
    score = score_text_local(row['text'], row['ticker'], model_path)
    return score

scores_series_local = stock_mentions.head(100).apply(score_row_local, axis=1)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

...

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [59]:
reduced_stock_mentions['score_local_no_finetuning'] = scores_series_local

In [60]:
reduced_stock_mentions[['ticker', 'text', 'score', 'score_local', 'score_local_no_finetuning']]

,ticker,text,score,score_local,score_local_no_finetuning
0,FAST,$INO rising fast,1,0,0
1,TECH,Best TECH ETF?,0,0,0
2,TECH,EqualOcean Presents: China Big Tech Crackdown ...,0,0,0
3,COST,Delta Airlines wants to increase its unvaccina...,0,0,0
4,TECH,Analysis into whether certain Chinese tech sto...,0,0,0
...,...,...,...,...,...
95,META,META Summarized News for the week [Feb 19],0,0,0
96,META,The Meta triangle wedge is finishing up,0,0,0
97,AXON,AXON,0,0,0
98,COST,Cost of Capital: Charlie Munger's Sarcastic Ta...,0,0,0


Here we check that the tokens used for every request are relatively low. This means that for the API we only need to worry about request rate and not token rate.

In [22]:
client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction='reply to the question and be friendly'),
    contents='how many days in a year?'
    )

response

NameError: name 'genai' is not defined

In [32]:
response.usage_metadata.total_token_count

42

legacy code:

In [16]:
from mlx_lm import load, generate

model, tokenizer = load("mlx-community/Llama-3.2-3B-Instruct-4bit")

stock = 'GME'

messages = [{
     'role': 'system',
     'content': f'''
                You need to act as an expert linguist, trader and researcher, read the reddit comment about {stock} that is given to you, and classify wether the comment is bullish or bearish about {stock}.
                Your answer should be 1 if the comment is positive about the stock, -1 if it is negative, and 0 if it is neutral. Do not reply to the comment, just classify it.
                ''',
   },
   {
     'role': 'user',
     'content': 'I think GME is going up despite the market being bearish',
   }]
prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True
)

text = generate(model, tokenizer, prompt=prompt, verbose=True)

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 96791.63it/s]


0
Prompt: 132 tokens, 355.723 tokens-per-sec
Generation: 2 tokens, 12.702 tokens-per-sec
Peak memory: 3.615 GB
